# TV Script Generation
In this project, I am generating [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  I have used the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons of the hit TV Series.  The Neural Network will generate a new TV script which is as long as a typical Simpsons scene.
## Getting the Data
I have used the entire original dataset.  It consists scenes of a variety of scenes from Simpsons' episodes (about 180K lines of text in total)

In [1]:
import helper

data_dir = 'data/simpsons-lines.txt'
text = helper.load_data(data_dir)

In [2]:
# Display some random text for sanity check
text[50:200]

". Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.\n\nLisa_Simpson: (NEAR TEARS) W"

## Exploring the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 139663
Number of scenes: 21592
Average number of sentences in each scene: 6.330122267506484
Number of lines: 158272
Average number of words in each line: 10.767703699959563

The sentences 0 to 10:
Miss_Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.

Lisa_Simpson: (NEAR TEARS) Where's Mr. Bergstrom?
Miss_Hoover: I don't know. Although I'd sure like to talk to him. He didn't touch my lesson plan. What did he teach you?
Lisa_Simpson: That life is worth living.
Edna Krabappel-Flanders: The polls will be open from now until the end of recess. Now, (SOUR) just in case any of you have decided to put any thought into this, we'll have our final statements. Martin?
Martin_Prince: (HOARSE WHISPER) I don't think there's anything left to say.
Edna Krabappel-Flanders: Bart?
Bart_Simpson: Victory party under the slide!
(Apartment_Buil

## Implement Preprocessing Functions
Now we'll implement the following preprocessing functions:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, I create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

These dictionaries are returned in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word:ii for ii, word in int_to_vocab.items()}
    return vocab_to_int, int_to_vocab


tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

The function `token_lookup` returns a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  We now create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. We need to make sure we don't use a token that could be confused as a word. Instead of using the token "dash", we'll use something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    lookup_dict = {}
    lookup_dict.update({'.' : '<PERIOD>'})
    lookup_dict.update({',' : '<COMMA>'})
    lookup_dict.update({'"' : '<QUOTATION_MARK>'})
    lookup_dict.update({';' : '<SEMICOLON>'})
    lookup_dict.update({'!' : '<EXCLAMATION_MARK>'})
    lookup_dict.update({'?' : '<QUESTION_MARK>'})
    lookup_dict.update({'(' : '<LEFT_PAREN>'})
    lookup_dict.update({')' : '<RIGHT_PAREN>'})
    lookup_dict.update({'--' : '<HYPHENS>'})
    lookup_dict.update({'\n' : '<NEW_LINE>'})
    return lookup_dict


tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
We'll save this checkpoint so that we can start from here after a restart. The preprocessed data has been saved to disk.

In [7]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [8]:
vocab_to_int['homer_simpson:']

12

## Building the Neural Network
Now we will build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [ ]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

### Input
The `get_inputs()` creates TF Placeholders for the Neural Network.  It will create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

The placeholders are returned in the following tuple `(Input, Targets, LearningRate)`

In [10]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32, [None, None], name = 'input')
    targets = tf.placeholder(tf.int32, [None, None], name = 'targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return inputs, targets, learning_rate


tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
We'll stack [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size will be set using `rnn_size`
- Cell State will be initialized using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - We will apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

The cell and initial state will be returned in the following tuple `(Cell, InitialState)`

In [12]:
def get_init_cell(batch_size, rnn_size, keep_prob=0.8, layers=3):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    def build_cell(rnn_size):
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        return lstm
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([cell] * layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    return cell, initial_state

tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [14]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.truncated_normal([vocab_size, embed_dim], stddev=0.05))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed

tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
We have created a RNN Cell in the `get_init_cell()` function.  Now it's time to use the cell to create a RNN.
- We'll build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - We will apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

The outputs and final_state state will be returned in the following tuple `(Outputs, FinalState)` 

In [16]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, final_state = tf.nn.dynamic_rnn(cell=cell,
                                   inputs=inputs,
                                   dtype=tf.float32)
    final_state = tf.identity(final_state, name="final_state")
    return outputs, final_state

tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Now we will
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

The logits and final state will be returned in the following tuple (Logits, FinalState) 

In [17]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embed = get_embed(input_data, vocab_size, rnn_size)
    outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None, weights_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.05,seed=None))
    return logits, final_state

tests.test_build_nn(build_nn)

Tests Passed


### Batches
The function `get_batches` implements functionality to create batches of input and targets using `int_text`.  The batches are a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If we can't fill the last batch with enough data, we'll drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [16]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    num_batches = len(int_text) // (batch_size  * seq_length)
    np_text = np.array(int_text[:num_batches * (batch_size  * seq_length)])
    in_text = np_text.reshape(-1, seq_length)
    tar_text = np.roll(np_text, -1).reshape(-1, seq_length)
    output = np.zeros(shape=(num_batches, 2, batch_size, seq_length), dtype=np.int)
    for idx in range(0, in_text.shape[0]):
        jj = idx % num_batches
        ii = idx // num_batches
        output[jj,0,ii,:] = in_text[idx,:]
        output[jj,1,ii,:] = tar_text[idx,:]
    return output


tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Now we will be tuning the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 1024
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 32
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 50

save_dir = './save'

### Build the Graph
Let's build the graph using the neural network we just implemented.

In [75]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Now we'll train the neural network on the preprocessed data.

In [76]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))
        saver = tf.train.Saver()
        saver.save(sess, save_dir)

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/149   train_loss = 11.010
Epoch   0 Batch   50/149   train_loss = 6.676
Epoch   0 Batch  100/149   train_loss = 6.494
Epoch   1 Batch    1/149   train_loss = 6.451
Epoch   1 Batch   51/149   train_loss = 5.949
Epoch   1 Batch  101/149   train_loss = 5.491
Epoch   2 Batch    2/149   train_loss = 5.073
Epoch   2 Batch   52/149   train_loss = 4.916
Epoch   2 Batch  102/149   train_loss = 4.775
Epoch   3 Batch    3/149   train_loss = 4.640
Epoch   3 Batch   53/149   train_loss = 4.632
Epoch   3 Batch  103/149   train_loss = 4.539
Epoch   4 Batch    4/149   train_loss = 4.490
Epoch   4 Batch   54/149   train_loss = 4.423
Epoch   4 Batch  104/149   train_loss = 4.393
Epoch   5 Batch    5/149   train_loss = 4.327
Epoch   5 Batch   55/149   train_loss = 4.283
Epoch   5 Batch  105/149   train_loss = 4.199
Epoch   6 Batch    6/149   train_loss = 4.168
Epoch   6 Batch   56/149   train_loss = 4.136
Epoch   6 Batch  106/149   train_loss = 4.100
Epoch   7 Batch    7/149   train_

Epoch  60 Batch   10/149   train_loss = 2.876
Epoch  60 Batch   60/149   train_loss = 2.900
Epoch  60 Batch  110/149   train_loss = 2.877
Epoch  61 Batch   11/149   train_loss = 2.861
Epoch  61 Batch   61/149   train_loss = 2.907
Epoch  61 Batch  111/149   train_loss = 2.904
Epoch  62 Batch   12/149   train_loss = 2.901
Epoch  62 Batch   62/149   train_loss = 2.867
Epoch  62 Batch  112/149   train_loss = 2.891
Epoch  63 Batch   13/149   train_loss = 2.851
Epoch  63 Batch   63/149   train_loss = 2.884
Epoch  63 Batch  113/149   train_loss = 2.850
Epoch  64 Batch   14/149   train_loss = 2.842
Epoch  64 Batch   64/149   train_loss = 2.869
Epoch  64 Batch  114/149   train_loss = 2.838
Epoch  65 Batch   15/149   train_loss = 2.895
Epoch  65 Batch   65/149   train_loss = 2.922
Epoch  65 Batch  115/149   train_loss = 2.856
Epoch  66 Batch   16/149   train_loss = 2.855
Epoch  66 Batch   66/149   train_loss = 2.830
Epoch  66 Batch  116/149   train_loss = 2.868
Epoch  67 Batch   17/149   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [77]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [78]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

The tensors are returned in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [79]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state = loaded_graph.get_tensor_by_name("final_state:0")
    probs = loaded_graph.get_tensor_by_name("probs:0")
    return input_tensor, initial_state, final_state, probs

tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
We'll now implement `pick_word()` function to select the next word using `probabilities`.

In [80]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    max_probability = np.argmax(probabilities)
    next_word = int_to_vocab[max_probability]
    return next_word

tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [83]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'barney_gumble'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
barney_gumble:(to rafter) c'mon, stool!
(simpson_home: int. simpson house - living room - continuous)

marge_simpson:(stricken noise)
homer_simpson:(moans)
homer_simpson:(pained sound)
homer_simpson:(chuckles) stupid clog!
homer_simpson:(defiant) look at this place!
homer_simpson:(puzzled) what?!
homer_simpson:(chuckling) i don't know.
homer_simpson:(accusing) i love you!
marge_simpson:(chanting) substantially-less...
homer_simpson:(dreamy)
(flanders_home: int. flanders house - living room - continuous)

homer_simpson:(exhausted) i don't understand. i have a problem early.
marge_simpson:(frustrated) oh, homie, i don't know.
homer_simpson:(stoic) i guess i can steal it.
bashir:(to kemi) but, uh, i have a plan.
(simpson_home: int. simpson house - kitchen - day)

homer_simpson:(moans)
marge_simpson:(blurting) homer, i don't want to be able to see you
